In [1]:
import tensorflow as tf

In [2]:
import recommendation
import word2vec
from  sentiment_model import *


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

In [3]:
# utagger를 설치 후, 사용 가능
import utagger_tokenize

프로그램 시작
python call utagger function
C:\Users\klplab\2021-1\Hlxcfg.txt


In [4]:
recom = recommendation.Recommend()
w2v = word2vec.Word2Vec()

In [5]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
modelName = "./model/sentiment-LSTM_model.ckpt"
saver.restore(sess, modelName)

INFO:tensorflow:Restoring parameters from ./model/sentiment-LSTM_model.ckpt


In [6]:
def grade(sentence):
    tokens = utagger_tokenize.tokenize(sentence)
    print(tokens)
    
    seq_length = [len(tokens)] + [0]*(batch_size-1)
    word_embedding = w2v.corpus2vec('./model/movie_word2vec300.model', [tokens])
    word_padded = w2v.zero_padding(word_embedding, batch_size, max_seq_length, vector_size)

    global sess
    result =  sess.run(prediction, feed_dict = {X: word_padded, seq_len:seq_length, keep_prob:1.0} ) 
        
    return result[0][0], result[0][1] #부정확률, 긍정확률

In [7]:
grade('시리즈를 마무리하는 좋은 영화') #-> 9.27점으로 취급

['시리즈/NNG', '를/JKO', '마무리하/VV', '는/ETM', '좋__01/VA', '은/ETM', '영화__01/NNG']


(0.07274156, 0.9272585)

In [8]:
new_critics = {}

In [9]:
while(1):
    print('관람하신 영화 제목을 입력하시오(종료:q)')
    input_ = input()
    if input_=='q':
        break
    search_result = recom.title_search(input_)
    
    if len(search_result)>1:
        print('\n검색 결과\n-----------------')
        for n, title in enumerate(search_result):
            print(n+1, ')', title)
            
        print('------------------------------')
        print('일치하는 제목에 해당하는 번호를 입력하시오')
        num = int(input())
        selected = search_result[num-1]
        
    elif len(search_result)==0:
        print('검색 실패, 다시 입력해주세요')
        continue
    
    else:
        num = 1
        selected = search_result[0]
        
    print('선택 결과 => ', selected)
    print('------------------------------')
    review = input('관람평을 입력하시오 : ')
    if len(review)<1:
        break
    else:
        score = grade(review)[1] * 10 #긍정일 확률을 점수로 활용
        new_critics[search_result[num-1]] = score
        print(score)
    print()

관람하신 영화 제목을 입력하시오(종료:q)
시빌ㄹ워
선택 결과 =>  캡틴 아메리카:시빌워
------------------------------
관람평을 입력하시오 : 내가 왜 이걸 이제서야 봤을까
['내__04/NP', '가/JKS', '왜__02/MAG', '이것/NP', 'ㄹ/JKO', '이제__01/NNG', '서/JKB', '야/JX', '보__01/VV', '았/EP', '을까/EC']
9.524945616722107

관람하신 영화 제목을 입력하시오(종료:q)
인시디어스
선택 결과 =>  인시디어스
------------------------------
관람평을 입력하시오 : 내가 이걸 왜 봤을까
['내__04/NP', '가/JKS', '이것/NP', 'ㄹ/JKO', '왜__02/MAG', '보__01/VV', '았/EP', '을까/EC']
0.5039110407233238

관람하신 영화 제목을 입력하시오(종료:q)
변호인
선택 결과 =>  변호인
------------------------------
관람평을 입력하시오 : 좋은 영화는 맞는데 내 취향은...
['좋__01/VA', '은/ETM', '영화__01/NNG', '는/JX', '맞__01/VV', '는데/EC', '나__03/NP', '의/JKG', '취향__01/NNG', '은/JX', './SF', './SF', './SF']
0.6451400369405746

관람하신 영화 제목을 입력하시오(종료:q)
해리포터

검색 결과
-----------------
1 ) 해리포터와 마법사의 돌
2 ) 해리포터와 비밀의 방
3 ) 해리포터와 아즈카반의 죄수
4 ) 해리포터와 죽음의 성물
------------------------------
일치하는 제목에 해당하는 번호를 입력하시오
4
선택 결과 =>  해리포터와 죽음의 성물
------------------------------
관람평을 입력하시오 : 시리즈를 마무리하는 좋은 영화
['시리즈/NNG', '를/JKO', '마무리하/VV'

In [10]:
#입력된 결과
new_critics

{'캡틴 아메리카:시빌워': 9.524945616722107,
 '인시디어스': 0.5039110407233238,
 '변호인': 0.6451400369405746,
 '해리포터와 죽음의 성물': 9.272584915161133,
 '라라랜드': 9.948849081993103,
 '위대한 쇼맨': 8.857707977294922,
 '신과함께:죄와벌': 5.843023061752319}

In [11]:
# 새로운 사용자에 대한 정보를 추가하고, 입력된 결과를 바탕으로 추천
recom.critics['new'] = new_critics
        
print('비슷한 취향의 사용자가 좋아한 영화')
print(recom.pearson_recommendation('new'))
print()

print('내가 좋아한 영화와 비슷한 유형의 영화')
print(recom.cosine_recommendation('new'))

비슷한 취향의 사용자가 좋아한 영화
[(10.0, '해리포터와 마법사의 돌'), (9.3, '겨울왕국2'), (9.25, '국제시장'), (9.1, '신비한 동물사전'), (8.795086136152777, '신과함께:인과연')]

내가 좋아한 영화와 비슷한 유형의 영화
['어벤져스:엔드게임', '어벤져스', '캡틴 아메리카:윈터솔져', '어벤져스:인피니티워', '가디언즈 오브 갤럭시 VOL.2', '캡틴 아메리카:퍼스트 어벤져', '해리포터와 비밀의 방', '해리포터와 아즈카반의 죄수', '리얼스틸']


C:\Users\klplab\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\klplab\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
